# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
import json
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv("output_data/cities.csv", encoding="utf-8")
cities.head()

,City,Country,Lat,Lng,Date,Cloudiness,Humidity,Max Temp,Wind Speed
0,lebu,CL,-37.616669,-73.650002,1603249200,5,78,49.26,16.80
1,havoysund,NO,70.996338,24.662170,1603249200,20,83,36.05,6.31
2,cabo san lucas,MX,22.890881,-109.912376,1603249200,3,75,80.87,8.72
3,codrington,AU,-38.266670,141.966675,1603249200,21,70,58.50,8.23
4,ushuaia,AR,-54.799999,-68.300003,1603249200,69,81,37.85,5.12


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
humidity = cities["Humidity"].astype(float)
humidity_max = humidity.max()
locations = cities[["Lat", "Lng"]]

In [7]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=humidity_max,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
narrowed_city_df = cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80) & (cities["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

,City,Country,Lat,Lng,Date,Cloudiness,Humidity,Max Temp,Wind Speed
0,arraial do cabo,BR,-22.966110,-42.027779,1603249200,0,84,72.59,19.66
1,taoudenni,ML,22.678329,-3.983610,1603249200,0,19,79.09,10.16
2,nokaneng,BW,-19.666670,22.266670,1603249200,0,21,71.24,7.85
3,grand gaube,MU,-20.006390,57.660831,1603249200,0,72,75.60,18.16
4,gojra,PK,31.148741,72.686562,1603249200,0,21,74.52,4.54


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_list = []

for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Lat']
    lng = narrowed_city_df.loc[i]['Lng']
    
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types": "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    response_json = response.json()
    try:
        hotel_list.append(response_json["results"][0]["name"])
    except:
        hotel_list.append('')
narrowed_city_df["Hotel Name"] = hotel_list
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,City,Country,Lat,Lng,Date,Cloudiness,Humidity,Max Temp,Wind Speed,Hotel Name
0,arraial do cabo,BR,-22.966110,-42.027779,1603249200,0,84,72.59,19.66,Cabo Frio
1,taoudenni,ML,22.678329,-3.983610,1603249200,0,19,79.09,10.16,Taoudenni
2,nokaneng,BW,-19.666670,22.266670,1603249200,0,21,71.24,7.85,
3,grand gaube,MU,-20.006390,57.660831,1603249200,0,72,75.60,18.16,Goodlands
4,gojra,PK,31.148741,72.686562,1603249200,0,21,74.52,4.54,Gojra


In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

AttributeError: 'list' object has no attribute 'iterrows'

In [21]:
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))